# TF-IDF를 활용한 Passage Retrieval 실습

### Requirements

In [ ]:
!pip install datasets
!pip install transformers

## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [1]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Dataset squad_kor_v1 downloaded and prepared to /Users/miyoungko/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/92f88eedc7d67b3f38389e8682eabe68caa450442cc4f7370a27873dbc045fe4. Subsequent calls will reuse this data.


In [2]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [3]:
tokenizer_func = lambda x: x.split(' ')

In [4]:
tokenizer_func(corpus[0])[:10]

['이로써', '레닌의', '두', '번째', '망명은', '1917년까지', '이어진다.', '레닌은', '망명기간', '내내']

## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [6]:
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

/Users/miyoungko/miniforge3/envs/py3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [7]:
sp_matrix.shape

(9606, 1272768)

In [8]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

                     TF-IDF
레닌은                0.215521
부르주아               0.146836
레닌은 세계관            0.089521
당으로                0.089521
근본적으로 평가함으로써       0.089521
발표하였는데, 이          0.089521
레닌은 망명기간           0.089521
볼셰비키 내부            0.089521
'유물론과 경험론'(1908)을  0.089521
'유물론과              0.089521


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [9]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

Query를 tf-idf vector로 변환

In [10]:
query_vec = vectorizer.transform([query])

In [11]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [12]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [13]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [14]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([5598, 5174, 7379], dtype=int32))

In [18]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 

## Wikipedia documents에 대해 TF-IDF 실습하기

In [20]:
# First load wikipedia dump
import json

# TODO: Write your own path
dump_path = 'data/wikipedia_documents.json'
with open(dump_path, 'r') as f:
    wiki = json.load(f)

In [25]:
len(wiki), type(wiki)

(60613, dict)

In [24]:
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [28]:
# TODO: Generate corpus and Train wiki_vectorizer on wikipedia_documents.json
corpus = [wiki[key]['text'] for key in wiki.keys()]
wiki_vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))
wiki_vectorizer.fit(corpus)
wiki_matrix = wiki_vectorizer.transform(corpus)


In [29]:
# Answer: (60613, 8719114) (num_doc, num_entity)
wiki_matrix.shape

(60613, 8719114)

In [33]:
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
query_vec = wiki_vectorizer.transform([query])

In [34]:
result = query_vec * wiki_matrix.T
result.shape

(1, 60613)

In [35]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([5598, 5174, 7379], dtype=int32))

In [36]:
print("[Search query]\n", query, "\n")


for i in range(k):
    print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
    doc_id = doc_ids[i]
    print(corpus[doc_id], "\n")

[Search query]
 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 

Top-1 passage with score 0.1899
조선인민공화국\n 그리하여 9월 6일에 '전국인민대표자회의'를 열고 박헌영이 건준을 「조선인민공화국」(약칭 인공)으로 변모시켰다 \n\n1945년 8월 하순, 여운형 등은 밀사 손치웅을 평양에 파견하여 조만식에게 남으로 내려올 것을 권고하였다. 그러나 조만식은 '여운형에게 뜻은 함께 하겠으나 몸은 여기 남겠다.'며 사양하였다. 1945년 9월 7일 여운형은 미군정이 시작되기 전까지 지속되었던 조선인민공화국의 주석에 선출된 이승만과 함께 부주석에 선출되었다. 9월 8일 미 24군단이 한반도 입성하고 군정을 선포하였을 때, 미군정장관 아널드는 오긍선을 만나는데, 오긍선은 한민당의 송진우를 소개해 주었다. 그리고 다음날 9월 11일 한민당을 대표한 조병옥, 윤보선 등은 미 군정장관 등을 만나 인공은 "일본과 협력한 한인집단"에 의해 조직되었으며, 여운형은 "한인들에게 잘 알려진 부일협력 정치인"이라고 왜곡하여 주장하였다. 이묘묵도 명월관에서 미군정 관리들에게 '여운형이 잘 알려진 친일파이며 인공은 공산주의적 경향이 있다.'고 왜곡하여 주장하였다 \n\n \n1945년 10월초, 여운형은 미 군정 사령관 존 하지와 첫 대면이 이루어졌는데, 첫 대화내용은 다음과 같다.\n\n이러한, 존 하지의 왜곡된 선입견과 그 배경은 미군정 고문으로 위촉된 9명의 한국민주당들의 모함으로 밝혀지게 된다\n\n \n1945년 10월 20일, 미군정의 주관하에 대대적인 환영행사 속에 이승만이 귀국하게 된다. 여운형은 이승만에게 찾아가 인공이 설립된 경로를 설명하고 인공 주석에 취임할 것을 요청하러 만났으나, 이승만은 침묵으로 답하고 이후 한국민주당과 함께 손을 잡고 독자적인 행보를 걷게 된다.\n이어 1945년 11월 3일, 대한민국 임시정부의 환국 때 그는 중경 임시정부 요인들을 맞아주었으나, 충칭 임시정부만을 추대하자는 내용인 '임정 정통